In [54]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np


pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 100

import warnings
warnings.filterwarnings('ignore')

from tabula import read_pdf
import camelot
from camelot import utils

import sys
import os
import shutil
import glob
from pathlib import Path

from paths import *
from scpts import manipulation
from scpts import io
from scpts import censo2010


from basedosdados import Storage, Table, Dataset


import yaml
import ruamel.yaml
import re


# Road Map

- First Download all tables from:
    - https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html
        - censos
            - censo_demografico_2010
                - resultados_do_universo
                    - agregados_por_setores_censitarios
- Put the .zip files in this path: "../data/censo/bases/original/fonte/"

- Run **CONVERT MAP TABLES** then **Organize directories**

## Organize directories

In [3]:
datasets_path = "../data/censo/bases/tratado/organized"
original_data_path = "../data/censo/bases/original/fonte/"

file_table_reference = censo2010.file_table_reference()
censo2010.create_dataset_folders(file_table_reference,datasets_path)


In [4]:
### Unzip and Organize directories
zip_files = [str(p) for p in Path(original_data_path).glob("*.zip")]

for zip_file in zip_files:
    f = ZipFile(zip_file)
    f.extractall(path=original_data_path)

    xls_files = [
        str(p) for p in Path(original_data_path).glob("**/*") if ".xls" in str(p)
    ]

    XLS_files = [
        str(p) for p in Path(original_data_path).glob("**/*") if ".XLS" in str(p)
    ]

    xls_files = xls_files + XLS_files

    censo2010.rename_and_move_files(xls_files)

    folder = [
        str(p) for p in Path(original_data_path).glob("*") if "zip" not in str(p)
    ][0]
    shutil.rmtree(folder)
    print(folder)

shutil.copytree(f"{datasets_path}/basico", f"{datasets_path}/basico_diretorio")

../data/censo/bases/original/fonte/AC
../data/censo/bases/original/fonte/AL
../data/censo/bases/original/fonte/AM
../data/censo/bases/original/fonte/AP
../data/censo/bases/original/fonte/BA
../data/censo/bases/original/fonte/CE
../data/censo/bases/original/fonte/DF
../data/censo/bases/original/fonte/Base informaçoes setores2010 universo ES
../data/censo/bases/original/fonte/GO
../data/censo/bases/original/fonte/MA
../data/censo/bases/original/fonte/MG
../data/censo/bases/original/fonte/MS
../data/censo/bases/original/fonte/MT
../data/censo/bases/original/fonte/PA
../data/censo/bases/original/fonte/PB
../data/censo/bases/original/fonte/PE_20171016
../data/censo/bases/original/fonte/PI
../data/censo/bases/original/fonte/PR
../data/censo/bases/original/fonte/RJ
../data/censo/bases/original/fonte/RN
../data/censo/bases/original/fonte/RO
../data/censo/bases/original/fonte/RR
../data/censo/bases/original/fonte/RS_20150527
../data/censo/bases/original/fonte/SC
../data/censo/bases/original/fon

'../data/censo/bases/tratado/organized/basico_diretorio'

### Padronize Tables

In [32]:
folders = glob.glob("../data/censo/bases/tratado/organized/*/")

In [38]:
for folder in folders:
    files = glob.glob(f'{folder}*.xls')
    print(folder)
    for file in files:

        uf = file[-6:][:2].upper()
        if uf == 'P1':
            uf='SP1'
        if uf == 'P2':
            uf='SP2'
        
        treated_path = f'{folder}sigla_uf={uf}'
        if os.path.isdir(treated_path) == False:
            df = pd.read_excel(file)
            df.columns = manipulation.normalize_cols(df.columns)
            os.mkdir(treated_path)
                    
            file_name = file.split('/')[-1].split('_')[0].lower() + '.xls'
            
            
            drop_col = [i for i in df.columns.tolist() if i.startswith('unnamed')]
#             drop_col = drop_col
            df = df.drop(columns=drop_col)
            
#             cols = df.columns.tolist()[:1] + ['uf_sigla'] + df.columns.tolist()[1:]
#             df['uf_sigla']=uf
            cols_rename = [col  for col in df.columns.tolist() if  col[-1]=='_']
            rename_dict = ref_dict = dict(zip(cols_rename, [col[:-1] for col in cols_rename]))
            df = df.rename(columns=rename_dict)
            
            for col in df.columns.tolist():
                df[col] = df[col].astype(str).str.replace(',','.')
#             display(df.head(2))
            df = df.rename(columns = {'cod_setor':'id_setor_censitario'})    
            if folder == '../data/censo/bases/tratado/organized/basico/':
                df = df.rename(columns = {'cod_setor':'id_setor_censitario'})
                cols = ['id_setor_censitario',
                        'v001','v002','v003','v004','v005','v006',
                        'v007','v008','v009','v010','v011','v012',
                       ]
                df = df[cols]

            elif  folder == '../data/censo/bases/tratado/organized/basico_diretorio/':
                df = df.rename(columns = {'cod_setor':'id_setor_censitario'})
                df = df.rename(columns = {
                    'cod_rm':'id_rm',
                    'cod_municipio':'id_municipio',
                    'cod_distrito':'id_distrito',
                    'cod_subdistrito':'id_subdistrito',
                    'cod_bairro':'id_bairro',
                    'nome_da_rm':'nome_rm',
                    'nome_do_distrito':'nome_distrito',
                    'nome_do_subdistrito':'nome_subdistrito',
                    'nome_do_bairro':'nome_bairro',
                })
                
                cols = ['id_setor_censitario','id_municipio',
                        'id_rm','nome_rm','id_distrito','nome_distrito',
                        'id_subdistrito','nome_subdistrito','id_bairro',
                        'nome_bairro','situacao_setor','tipo_setor']
                df = df[cols]
                df['nome_bairro'] = df['nome_bairro'].str.replace('todos os setores','')\
                                                     .str.replace(r"\(.*\)",'').str.strip()
                
            else:
                cols = [col for col in df.columns.tolist() if re.match('v\d\d\d', col)]
                df = df[['id_setor_censitario'] + cols]
    
        else:
            pass
        f_name = file_name.split('.')[0]
        print(uf,': ',df.shape)
        df.to_csv(f'{treated_path}/{f_name}.csv', index=False, encoding='utf-8')
        os.remove(file)

    print(' ')

../data/censo/bases/tratado/organized/alfabetizacao_homens_mulheres/
 
../data/censo/bases/tratado/organized/alfabetizacao_total/
 
../data/censo/bases/tratado/organized/basico/
 
../data/censo/bases/tratado/organized/basico_diretorio/
 
../data/censo/bases/tratado/organized/domicilio_caracteristicas_gerais/
 
../data/censo/bases/tratado/organized/domicilio_moradores/
 
../data/censo/bases/tratado/organized/domicilio_renda/
 
../data/censo/bases/tratado/organized/entorno/
 
../data/censo/bases/tratado/organized/entorno01/
 
../data/censo/bases/tratado/organized/entorno02/
 
../data/censo/bases/tratado/organized/entorno03/
 
../data/censo/bases/tratado/organized/entorno04/
 
../data/censo/bases/tratado/organized/entorno05/
ES :  (6380, 221)
 
../data/censo/bases/tratado/organized/idade_homens/
 
../data/censo/bases/tratado/organized/idade_mulheres/
 
../data/censo/bases/tratado/organized/idade_total/
 
../data/censo/bases/tratado/organized/pessoa_renda/
 
../data/censo/bases/tratado/org

## Merge Entorno* Tables

In [45]:
partitions = [partition.split('/')[-1] for partition \
              in glob.glob("../data/censo/bases/tratado/organized/entorno01/*")]

# from functools import reduce

partitions_done = [partition.split('/')[-1] for partition \
              in glob.glob("../data/censo/bases/tratado/organized/entorno/*")]
partitions_f = [pt for pt in partitions if pt not in partitions_done]

entorno_path = '../data/censo/bases/tratado/organized/entorno'
for partition in partitions_f:
    print(partition)
    e1 = pd.read_csv(f'{entorno_path}01/{partition}/entorno01.csv',dtype={'id_setor_censitario':int}).sort_values(by='id_setor_censitario')
    e2 = pd.read_csv(f'{entorno_path}02/{partition}/entorno02.csv',dtype={'id_setor_censitario':int}).sort_values(by='id_setor_censitario')
    e3 = pd.read_csv(f'{entorno_path}03/{partition}/entorno03.csv',dtype={'id_setor_censitario':int}).sort_values(by='id_setor_censitario')
    e4 = pd.read_csv(f'{entorno_path}04/{partition}/entorno04.csv',dtype={'id_setor_censitario':int}).sort_values(by='id_setor_censitario')
    e5 = pd.read_csv(f'{entorno_path}05/{partition}/entorno05.csv',dtype={'id_setor_censitario':int}).sort_values(by='id_setor_censitario')
    
    dfs = [e1,e2,e3,e4,e5]
    ef = reduce(lambda left,right: pd.merge(left,right,
                                            on='id_setor_censitario',
                                            how='outer'), dfs)
    
    
    print('e1:', e1.shape)
    print('e2:', e2.shape)
    print('e3:', e3.shape)
    print('e4:', e4.shape)
    print('e5:', e5.shape)
    print('ef:', ef.shape)
    print(' ')
    os.mkdir(f'{entorno_path}/{partition}')
    ef.to_csv(f'{entorno_path}/{partition}/entorno.csv', index=False)
    
    del dfs, e1, e2, e3, e4, e5, ef
    
## GO have some cols missing
go = pd.read_csv('../data/censo/bases/tratado/organized/entorno/sigla_uf=GO/entorno.csv')
es = pd.read_csv('../data/censo/bases/tratado/organized/entorno/sigla_uf=ES/entorno.csv')
goc = go.columns.tolist()
esc = es.columns.tolist()
create_cols = [col for col in esc if col not in goc]

for col in create_cols:
    go[col] = np.nan
go = go[esc]
go.to_csv('../data/censo/bases/tratado/organized/entorno/sigla_uf=GO/entorno.csv', index=False)


### Remove entorno* cols
shutil.rmtree('../data/censo/bases/tratado/organized/entorno01/')
shutil.rmtree('../data/censo/bases/tratado/organized/entorno02/')
shutil.rmtree('../data/censo/bases/tratado/organized/entorno03/')
shutil.rmtree('../data/censo/bases/tratado/organized/entorno04/')
shutil.rmtree('../data/censo/bases/tratado/organized/entorno05/')

## Upload To BD+

### Generate and fill configs

In [70]:
import basedosdados as bd

In [71]:
reference_table_path = '../data/censo/bases/tratado/reference_tables/'
folders = glob.glob("../data/censo/bases/tratado/organized/*/")
folders.remove( '../data/censo/bases/tratado/organized/basico_diretorio/')

In [72]:
# i=2
i=0
for folder in folders:
    print(i,' - ',folder)
    i+=1
    folder_name = folder.split('/')[-2]
#     reference_table_file = reference_table_path + folder_name + '.xlsx'
#     ref = pd.read_excel(reference_table_file)

#     bd_base_path = '/mnt/AEB0CCA7B0CC777D/Jlab/gabinete_sv/mais/bases/br_ibge_censo2010/' + f'{folder_name}_setor_censitario/table_config.yaml'
    
    
    
#     ref.columns = manipulation.normalize_cols(ref.columns)
    
#     if 'codigo_da_variavel' in ref.columns.tolist():
#         ref['codigo_da_variavel'] = manipulation.normalize_cols(ref['codigo_da_variavel'])

#         var_name = ref['codigo_da_variavel'].tolist()
#         var_desc = ref['descricao_do_setor'].tolist()
        
#     elif 'nome_da_variavel' in ref.columns.tolist():
#         ref['nome_da_variavel'] = manipulation.normalize_cols(ref['nome_da_variavel'])

#         var_name = ref['nome_da_variavel'].tolist()
#         var_desc = ref['descricao_da_variavel'].tolist()

#     ref_dict = dict(zip(var_name, var_desc))
#     ref_dict['uf_sigla'] = 'Sigla do Estado'
#     ref_dict['sigla_uf'] = 'Sigla do Estado'
#     ref_dict['situacao_setor'] = 'Código de situação do setor - Situação urbana - Códigos: 1, 2 e 3    - 1 - Área urbanizada de cidade ou vila    - 2 - Área não urbanizada de cidade ou vila    - 3 - Área urbana isolada - Situação rural – códigos: 4, 5, 6, 7 e 8    - 4 - Aglomerado rural de extensão urbana     - 5 - Aglomerado rural isolado – povoado    - 6 - Aglomerado rural isolado – núcleo    - 7 - Aglomerado rural isolado - outros aglomerados    - 8 - Zona rural, exclusive aglomerado rural'
    
    
    tb = Table(f'setor_censitario_{folder_name}_2010','br_ibge_censo_demografico')
    tb.create(f'../data/censo/bases/tratado/organized/{folder_name}/',
              partitioned=True,
              if_table_exists='replace',
              if_storage_data_exists='repalce',
              if_table_config_exists='pass',
             )

#     yaml =ml ruamel.yaml.YAML()
#     # yaml.preserve_quotes = Truehttps://basedosdados.org/api/3/action/tag_list
#     with open(bd_base_path) as fp:
#         config_yaml = yaml.load(fp)


#     for var_dict in config_yaml['columns']:
#             var_dict['description'] = ref_dict[var_dict['name']]
            
#     config_yaml['treated_by']['nome'] = 'Diego Oliveira'
#     config_yaml['treated_by']['website'] = 'https://www.al.sp.gov.br/servidor/lista/?todos=true'
#     config_yaml['treated_by']['email'] = 'ds.oliveira404@gmail.com'

#     config_yaml['published_by']['name'] = 'Diego Oliveira'
#     config_yaml['published_by']['website'] = 'github.com/d116626'
#     config_yaml['published_by']['email'] =  'ds.oliveira404@gmail.com'

#     config_yaml['treatment_description'] = 'Padronização do nome das colunas, remoção de textos e logos, unificação das bases'

#     config_yaml['data_update_frequency'] = '10 anos'

#     config_yaml['observation_level'] = ['cod_setor','sigla_uf' ]

#     config_yaml['coverage_geo'] = ['BR']


#     config_yaml['coverage_time'] = ['2010']

#     config_yaml['columns'] = [i for i in config_yaml['columns'][:1]] + [config_yaml['columns'][-1]] + [i for i in config_yaml['columns'][1:-1]]
    
#     with open(bd_base_path, 'w') as fp:
#         yaml.dump(config_yaml, fp)
        

    print(' ')


Uploading files:   0%|          | 0/28 [00:00<?, ?it/s]

0  -  ../data/censo/bases/tratado/organized/alfabetizacao_homens_mulheres/


Uploading files:   0%|          | 0/28 [00:00<?, ?it/s]

 
1  -  ../data/censo/bases/tratado/organized/alfabetizacao_total/


Uploading files:   0%|          | 0/28 [00:00<?, ?it/s]

...

Uploading files:   0%|          | 0/28 [00:00<?, ?it/s]

 
20  -  ../data/censo/bases/tratado/organized/responsavel_domicilios_mulheres/


Uploading files:   0%|          | 0/28 [00:00<?, ?it/s]

 
21  -  ../data/censo/bases/tratado/organized/responsavel_renda/


Uploading files: 100%|██████████| 28/28 [01:33<00:00,  3.33s/it]


## Publish

In [73]:
reference_table_path = '../data/censo/bases/tratado/reference_tables/'
folders = glob.glob("../data/censo/bases/tratado/organized/*/")
folders.remove( '../data/censo/bases/tratado/organized/basico_diretorio/')

In [75]:
i=0
for folder in folders:
    print(i,' - ',folder)
    i+=1
    folder_name = folder.split('/')[-2]
    
    tb = Table(f'setor_censitario_{folder_name}_2010','br_ibge_censo_demografico')
    tb.publish(if_exists='replace')

0  -  ../data/censo/bases/tratado/organized/alfabetizacao_homens_mulheres/
1  -  ../data/censo/bases/tratado/organized/alfabetizacao_total/
2  -  ../data/censo/bases/tratado/organized/basico/
3  -  ../data/censo/bases/tratado/organized/domicilio_caracteristicas_gerais/
4  -  ../data/censo/bases/tratado/organized/domicilio_moradores/
5  -  ../data/censo/bases/tratado/organized/domicilio_renda/
6  -  ../data/censo/bases/tratado/organized/entorno/
7  -  ../data/censo/bases/tratado/organized/idade_homens/
8  -  ../data/censo/bases/tratado/organized/idade_mulheres/
9  -  ../data/censo/bases/tratado/organized/idade_total/
10  -  ../data/censo/bases/tratado/organized/pessoa_renda/
11  -  ../data/censo/bases/tratado/organized/raca_alfabetizacao_idade_genero/
12  -  ../data/censo/bases/tratado/organized/raca_idade_0_4_genero/
13  -  ../data/censo/bases/tratado/organized/raca_idade_genero/
14  -  ../data/censo/bases/tratado/organized/registro_civil/
15  -  ../data/censo/bases/tratado/organized/r

## Add Setor Censitario

In [68]:
tb = Table(f'setor_censitario','br_bd_diretorios_brasil')

In [61]:
tb.create(f'../data/censo/bases/tratado/organized/basico_diretorio/',
          partitioned=True,
          if_table_exists='replace',
          if_storage_data_exists='replace',
          if_table_config_exists='raise',
         )

Uploading files: 100%|██████████| 28/28 [00:35<00:00,  1.28s/it]


In [69]:
tb.publish(if_exists='replace')

# CONVERT MAP TABLES

In [47]:
def extract_df(var,pages):
    ../data/censo/
    dd = camelot.read_pdf('../data/censo/map_tables/BASE DE INFORMAАхES POR SETOR CENSIT╡RIO Censo 2010 - Universo.pdf', pages=pages)
    df = pd.DataFrame()

    for i in range(len(dd)):
        df = pd.concat([df,dd[i].df], axis=0)

    df = df.reset_index(drop=True)
    for col in df.columns.to_list():
        df[col] = df[col].str.replace('\r',' ').str.replace('\n',' ')
    # df.columns = df.columns.str.replace('\r',' ')
    df.columns = ['Nome da variável','Descrição da variável']

    mask = df['Descrição da variável']!=df['Descrição da variável'][0]
    df = df[mask]
    
    mask = df['Descrição da variável']!='Descrição da variável'
    df = df[mask]
    
    return df

### Básico_UF.csv

In [247]:
var = 'Básico_UF'
pages='45-46'
df = extract_df(var,pages)

In [155]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Domicilio01_UF.csv

In [153]:
var = 'Domicilio01_UF'
pages = '47-58'
df = extract_df(var,pages)

In [155]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Domicilio02_UF.csv

In [156]:
var = 'Domicilio02_UF'
pages = '59-65'
df = extract_df(var,pages)

In [158]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Responsável01_UF.xls

In [159]:
var = 'Responsável01_UF'
pages = '66-70'
df = extract_df(var,pages)

In [161]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Responsável02_UF.csv

In [162]:
var = 'Responsável02_UF'
pages = '71-78'
df = extract_df(var,pages)

In [163]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa01_UF.csv

In [165]:
var = 'Pessoa01_UF'
pages = '79-81'
df = extract_df(var,pages)

In [167]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa02_UF.csv

In [168]:
var = 'Pessoa02_UF'
pages = '82-87'
df = extract_df(var,pages)

In [170]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa03_UF.csv

In [177]:
var = 'Pessoa03_UF'
pages='88-98'
df = extract_df(var,pages)

In [178]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa04_UF.xls

In [241]:
var = 'Pessoa04_UF'
pages = '99-105'
df = extract_df(var,pages)

In [242]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa05_UF.xls

In [237]:
var = 'Pessoa05_UF'
pages = '106'
df = extract_df(var,pages)

In [238]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa06_UF.xls

In [239]:
var = 'Pessoa06_UF'
pages = '107-116'
df = extract_df(var,pages)

In [240]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa07_UF.xls

In [182]:
var = 'Pessoa07_UF'
pages = '117-123'
df = extract_df(var,pages)

In [183]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa08_UF.xls

In [243]:
var = 'Pessoa08_UF'
pages = '124-131'
df = extract_df(var,pages)

In [244]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa09_UF.xls

In [245]:
var = 'Pessoa09_UF'
pages = '132-138'
df = extract_df(var,pages)

In [246]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa10_UF.xls

In [189]:
var = 'Pessoa10_UF'
pages = '139'
df = extract_df(var,pages)

In [191]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa13_UF.xls

In [192]:
var = 'Pessoa13_UF'
pages = '140-143'
df = extract_df(var,pages)

In [194]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa11_UF.xls

In [195]:
var = 'Pessoa11_UF'
pages = '144-147'
df = extract_df(var,pages)

In [197]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Pessoa12_UF.xls

In [198]:
var = 'Pessoa12_UF'
pages = '148-152'
df = extract_df(var,pages)

In [199]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### DomicílioRenda _UF.xls

In [200]:
var = 'DomicilioRenda _UF'
pages = '153'
df = extract_df(var,pages)

In [202]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### ResponsavelRenda_UF

In [203]:
var = 'ResponsavelRenda_UF'
pages = '154-162'
df = extract_df(var,pages)

In [205]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### PessoaRenda_UF.xls

In [206]:
var = 'PessoaRenda_UF'
pages = '163-171'
df = extract_df(var,pages)

In [209]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Entorno01_UF.xls

In [235]:
var = 'Entorno01_UF'
pages = '172-183'
df = extract_df(var,pages)

In [236]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Entorno02_UF.xls

In [214]:
var = 'Entorno02_UF'
pages = '184-196'
df = extract_df(var,pages)

In [216]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Entorno03_UF.xls

In [225]:
var = 'Entorno03_UF'
pages = '197-210'
df = extract_df(var,pages)

In [226]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Entorno04_UF.xls

In [229]:
var = 'Entorno04_UF'
pages = '211-223'
df = extract_df(var,pages)

In [231]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)

### Entorno05_UF.xls

In [232]:
var = 'Entorno05_UF'
pages = '224-234'
df = extract_df(var,pages)

In [234]:
df.to_excel(f'../data/censo/map_tables/{var}.xlsx', index=False)